In [1]:
import pandas as pd
import ast
import numpy as np
import sys
import geopandas
from geopy.distance import geodesic
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.ops import nearest_points
from shapely import wkt

### import data

In [5]:
properati = pd.read_csv('properati_data/combined_sell.csv', encoding= "ISO-8859-1")
prop_data = pd.read_csv('prop_data/cleaned_data.csv', encoding= "ISO-8859-1")
census = pd.read_csv('census/cleaned_data2.csv',encoding= "ISO-8859-1")

### Create GeoPandas dataframes

In [6]:
def modify_poly(data):
    coord_list = []
    #row = ast.literal_eval(data)
    for coord in data:
        coord_list.append(tuple(coord))
    return coord_list

In [7]:
# create a new column called coordinates on properati data frame
properati['coordinates'] = list(zip(properati.lon, properati.lat))
properati['coordinates'] = properati['coordinates'].apply(Point)

# convert properati data into geopandas dataframe
properati_gdf = geopandas.GeoDataFrame(properati, geometry='coordinates')

# convert the longitudes and latitudes in the properaty values dataframe into float (they were originally in string)
prop_data.Latitude = pd.to_numeric(prop_data.Latitude, errors='coerce')
prop_data.Longitude = pd.to_numeric(prop_data.Longitude, errors='coerce')

#print (properati_gdf.coordinates.head(5))
print (properati_gdf.coordinates.isnull().values.any())
print (len(properati_gdf))

# convert property values data into geopandas dataframe
prop_data['coordinates'] = list(zip(prop_data.Longitude, prop_data.Latitude))
#prop_data['coordinates'] = list(zip(prop_data.Latitude, prop_data.Longitude))
prop_data['coordinates'] = prop_data['coordinates'].apply(Point)
prop_data_gdf = geopandas.GeoDataFrame(prop_data, geometry='coordinates')


# convert census polygon data into tuples and polygon
census.coordinates = census.coordinates.apply(lambda x: ast.literal_eval(x)[0])
census.coordinates = census.coordinates.apply(lambda x: modify_poly(x))
census.coordinates = census.coordinates.apply(Polygon)

census_gdf = geopandas.GeoDataFrame(census, geometry='coordinates')
print (len(census_gdf))
#print (census_gdf.coordinates[0])
print (census_gdf.coordinates.isnull().values.any())

False
1871732
3552
False


### Filter only data that is in the city of the Buenos Aires

In [8]:
properati_census_gdf = geopandas.sjoin(properati_gdf, census_gdf, how="inner", op='within')
print ('{:.2f}%'.format(len(properati_census_gdf)/len(properati_gdf)))

print (len(properati_census_gdf))

/anaconda3/envs/geopandas/lib/python3.7/site-packages/numpy/lib/function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


0.25%
464869


#### save the properati/census combined data because the sjoin takes forever

In [3]:
temp_df = pd.DataFrame(properati_census_gdf)
temp_df.to_csv('properati_data/sell_census.csv')
temp_df = temp_df.drop_duplicates(subset='id_left')
print (prop_census_df.head(5))
properati_census_gdf = geopandas.GeoDataFrame(prop_census_df, geometry='coordinates')

NameError: name 'properati_census_gdf' is not defined

In [5]:
temp_df = pd.read_csv('properati_data/sell/sell_census.csv')
#print (temp_df.id_left.head(5))
temp_df = temp_df.drop_duplicates(subset='id_left')
print (len(temp_df))
temp_df.to_csv('properati_data/sell/sell_census_stripped.csv', columns=['id_left', 'lat', 'lon'])
temp_df.coordinates = list(zip(temp_df.lon, temp_df.lat))
temp_df.coordinates = temp_df.coordinates.apply(Point)
properati_census_gdf = geopandas.GeoDataFrame(temp_df, geometry='coordinates')
#print ('{:.2f}'.format((len(properati_census_gdf)/ 464869)*100))

116425


#### import barrios geojson

In [6]:
barrios = pd.read_csv('shape files/barrios.csv')
barrios.WKT = barrios.WKT.apply(wkt.loads)

barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VELEZ SARSFIELD', 'VELEZ SARFIELD'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VILLA GRAL. MITRE', 'VILLA GENERAL MITRE'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VILLA GRAL. MITR', 'VILLA GENERAL MITRE'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VILLA GRAL MITRE', 'VILLA GENERAL MITRE'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('VERSALLES', 'VERSAILLES'))))
barrios['barrio'] = (barrios['barrio'].apply(lambda x: str(x.replace('MONSERRAT', 'MONTSERRAT'))))

barrio_df = pd.read_csv('barrio_table.csv')
barrio_df['b_id'] = [int(barrio_df[barrio_df.Barrio == i].id) for i in barrio_df.Barrio]
barrio_df['barrio'] = barrio_df.Barrio

print (barrio_df['barrio'])

barrios = pd.merge(barrios, barrio_df, on='barrio', how='left')
df = barrios[barrios.id.isnull()]

barrios_gdf = geopandas.GeoDataFrame(barrios, geometry='WKT')

0                 PALERMO
1                BELGRANO
2                   NUÑEZ
3           VILLA ORTUZAR
4               BALVANERA
5           SAN CRISTOBAL
6            VILLA CRESPO
7     VILLA GENERAL MITRE
8                FLORESTA
9        PARQUE CHACABUCO
10      PARQUE AVELLANEDA
11              CABALLITO
12                ALMAGRO
13                POMPEYA
14          VILLA URQUIZA
15               SAAVEDRA
16       VILLA PUEYRREDON
17             VILLA LURO
18              MATADEROS
19                 FLORES
20             VERSAILLES
21                LINIERS
22         VELEZ SARFIELD
23           VILLA DEVOTO
24                   BOCA
25              AGRONOMIA
26              CHACARITA
27               PATERNAL
28             COLEGIALES
29       VILLA DEL PARQUE
30           VILLA LUGANO
31       VILLA SANTA RITA
32               BARRACAS
33           MONTE CASTRO
34        VILLA RIACHUELO
35                 RETIRO
36             MONTSERRAT
37          VILLA SOLDATI
38          

In [ ]:
properati_census_gdf = properati_census_gdf.drop(columns=['id_right', 'index_right'], axis=1)
print (properati_census_gdf.columns.values)
final_gdf = geopandas.sjoin(properati_census_gdf , barrios_gdf, how="inner", op='within')
print (final_gdf.columns.values)
print (len(final_gdf))


['Unnamed: 0' 'dataset_date' 'id_left' 'created_on' 'operation'
 'property_type' 'place_name' 'country_name' 'state_name' 'geonames_id'
 'lat_lon' 'lat' 'lon' 'price' 'currency' 'price_aprox_local_currency'
 'price_aprox_usd' 'coordinates' 'AREA' 'Commune' 'DEPTO' 'FRAC'
 'Computer Percent' 'Computer Quantile' 'Cellular Percent'
 'Cellular Quantile' 'Rent Percent' 'Rent Quantile' 'LINK'
 'Immigration Percent' 'Immigration Quantile' 'Education Percent'
 'Education Quantile' 'PERIMETER' 'Owner Percent' 'Owner Quantile' 'PROV'
 'RADIO' 'RADPAIS_' 'REDCODE' 'Regular Percent' 'Regular Quantile' 'TIPO'
 'Uninhabited Percent' 'Uninhabited Quantile' 'type']


In [ ]:
temp_df = pd.DataFrame(final_gdf)
df = temp_df[temp_df.b_id.isnull()]
print (df)
#temp_df.to_csv('properati_data/sell data/sell_census_barrios.csv')


In [ ]:
filtered_gdf = final_gdf[['dataset_date', 'created_on', 'operation', 'property_type', 'place_name',\
                          'state_name', 'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',\
                         'Commune', 'barrio','b_id', 'comuna', 'perimetro','area']]

filter_df = pd.DataFrame(filtered_gdf)
filter_df = filter_df[filter_df['price_aprox_local_currency'] > 0]

In [ ]:
barrios_filter_df = filter_df[['barrio','comuna','b_id', 'perimetro','area','price_aprox_local_currency', 'price_aprox_usd','created_on']]
grouped = barrios_filter_df.groupby(['b_id','barrio'], as_index=False).agg({"price_aprox_local_currency": ["min", "max", "mean", "std"], "price_aprox_usd": ["min", "max", "mean", "std"],\
                                                "created_on": "count"})
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
print (grouped.head(5))
grouped.to_csv('properati_data/sell/properati_sell_grouped.csv')

In [ ]:
grouped.to_json("properati_data/sell/properati_sell.json", orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)



In [ ]:
filter_df['created_on'] = pd.to_datetime(filter_df['created_on'])
filter_df['month'] = filter_df.created_on.apply(lambda x: x.month)
filter_df['year'] = filter_df.created_on.apply(lambda x: x.year)
filter_df = filter_df[filter_df['price_aprox_local_currency'] > 0]

barrios_filter_df = filter_df[['barrio','b_id','dataset_date','comuna','perimetro','area','price_aprox_local_currency', 'price_aprox_usd',\
                             'created_on', 'month', 'year']]
print (barrios_filter_df.head(5))

In [ ]:
grouped_month = barrios_filter_df.groupby(['barrio', 'b_id','month', 'year'], as_index=False).agg({"price_aprox_local_currency": ["min", "max", "mean", "std"], "price_aprox_usd": ["min", "max", "mean", "std"],\
                                                "created_on": "count"}).fillna(0) #for the std deviation values that resulted in NaN 
grouped_month.columns = ["_".join(x) for x in grouped_month.columns.ravel()]
print (grouped_month.head(5))
print (len(grouped_month))

In [ ]:
grouped_month.to_csv('properati_data/sell/properati_sell_grouped_month.csv')
grouped_month.to_json("properati_data/sell/properati_sell_month.json", orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

